In [1]:
import numpy as np
import json
from sklearn.decomposition import PCA
from scipy.stats import spearmanr
import os

# ===============================
# CONFIG
# ===============================

VIT_PATH = '/home/maria/ProjectionSort/data/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/ProjectionSort/data/hybrid_neural_responses_reduced.npy'
OUT_DIR = './data_dev'      # folder next to scatter.html
BEST_NEURON_INDEX = 20      # change to whichever neuron you want to visualize

# ===============================
# LOAD DATA
# ===============================

print("Loading data...")
vit_logits = np.load(VIT_PATH, allow_pickle=True)
X_vit = np.asarray(vit_logits['natural_scenes'])   # shape (118, 1000)

neural_activations = np.load(NEURAL_PATH)          # shape (39209, 118)

print("X_vit:", X_vit.shape)
print("neural:", neural_activations.shape)

# ===============================
# COMPUTE VIT PC1 SCORES
# ===============================

print("Computing PCA on ViT logits...")
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_vit)

scores_vit_full = X_pca[:, 0]     # 118-long array

print("PC1 scores computed.")

# ===============================
# SELECT NEURON AND COMPUTE ITS VECTOR
# ===============================

neuron_scores = neural_activations[BEST_NEURON_INDEX]

rho, p = spearmanr(scores_vit_full, neuron_scores)
print(f"Neuron {BEST_NEURON_INDEX} Spearman rho = {rho}, p = {p}")

# ===============================
# ENSURE OUTPUT DIR EXISTS
# ===============================

os.makedirs(OUT_DIR, exist_ok=True)

# ===============================
# EXPORT JSON FILES FOR D3.js
# ===============================

vit_file = os.path.join(OUT_DIR, "scores_vit_full.json")
neuron_file = os.path.join(OUT_DIR, "neuron_scores.json")

with open(vit_file, "w") as f:
    json.dump(scores_vit_full.tolist(), f, indent=2)

with open(neuron_file, "w") as f:
    json.dump(neuron_scores.tolist(), f, indent=2)

print(f"Saved {vit_file}")
print(f"Saved {neuron_file}")

# ===============================
# OPTIONAL: EXPORT ALL NEURON CORRELATIONS
# ===============================

print("Computing correlations for all neurons (optional)...")
correlations = []

for j in range(neural_activations.shape[0]):
    r, _ = spearmanr(scores_vit_full, neural_activations[j])
    correlations.append(r)

corr_file = os.path.join(OUT_DIR, "all_neuron_correlations.json")
with open(corr_file, "w") as f:
    json.dump(correlations, f, indent=2)

print(f"Saved {corr_file}")

print("Done.")


Loading data...
X_vit: (118, 1000)
neural: (39209, 118)
Computing PCA on ViT logits...
PC1 scores computed.
Neuron 20 Spearman rho = 0.1776943776103961, p = 0.054222308621634074
Saved ./data_dev/scores_vit_full.json
Saved ./data_dev/neuron_scores.json
Computing correlations for all neurons (optional)...
Saved ./data_dev/all_neuron_correlations.json
Done.
